# Medical Segmentation Decathlon: Ingestion & Normalization

MSD Task06_Lung: 63 training CT scans with lung tumor segmentation masks (~8.5GB). [Dataset reference](http://medicaldecathlon.com/)

**Workflow:**
1. Ingest raw NIfTIs (CT images + segmentation masks)
2. Transform with `map()` (resample to uniform spacing)
3. Write normalized collections (ML-ready)

In [ ]:
import sys

sys.path.insert(0, "..")

import subprocess
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tiledb
from config import MSD_LUNG_URI, S3_REGION

from radiobject.ctx import S3Config, SliceOrientation, TileConfig, configure, global_ctx
from radiobject.radi_object import RadiObject

# Define URIs for raw and normalized data
MSD_LUNG_RAW_URI = MSD_LUNG_URI.replace("radiobject-2mm", "radiobject-raw")

print(f"Raw URI: {MSD_LUNG_RAW_URI}")
print(f"Normalized URI: {MSD_LUNG_URI}")

In [ ]:
# Configure S3 access and tile layout
if MSD_LUNG_URI.startswith("s3://"):
    configure(s3=S3Config(region=S3_REGION, max_parallel_ops=8))
configure(tile=TileConfig(orientation=SliceOrientation.ISOTROPIC))

# Data paths
S3_NIFTI_URI = "s3://souzy-scratch/msd-lung/nifti"
DATA_DIR = Path("../data/msd_lung/Task06_Lung")
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Local data: {DATA_DIR.resolve()}")
print(f"Target: {MSD_LUNG_URI}")

In [ ]:
# Force re-ingestion: Set to True to delete existing data and re-ingest
FORCE_REINGEST = False


def uri_exists(uri: str) -> bool:
    try:
        radi = RadiObject(uri)
        _ = radi.collection_names
        return True
    except Exception:
        return False


def delete_uri(uri: str) -> None:
    """Delete TileDB group at URI (works for S3 and local)."""
    try:
        ctx = global_ctx()
        vfs = tiledb.VFS(ctx=ctx)
        if vfs.is_dir(uri):
            vfs.remove_dir(uri)
            print(f"Deleted: {uri}")
    except Exception as e:
        print(f"Could not delete {uri}: {e}")


if FORCE_REINGEST:
    print("Force re-ingestion enabled. Deleting existing data...")
    delete_uri(MSD_LUNG_RAW_URI)
    delete_uri(MSD_LUNG_URI)

SKIP_RAW_INGESTION = uri_exists(MSD_LUNG_RAW_URI) and not FORCE_REINGEST
SKIP_NORMALIZATION = uri_exists(MSD_LUNG_URI) and not FORCE_REINGEST

print(f"Skip raw ingestion: {SKIP_RAW_INGESTION}")
print(f"Skip normalization: {SKIP_NORMALIZATION}")

In [ ]:
if not SKIP_RAW_INGESTION and not (DATA_DIR / "imagesTr").exists():
    print(f"Downloading from {S3_NIFTI_URI}...")
    result = subprocess.run(
        ["aws", "s3", "sync", S3_NIFTI_URI, str(DATA_DIR)],
        capture_output=True,
        text=True,
    )
    if result.returncode != 0:
        raise RuntimeError(f"Download failed: {result.stderr}")
    print("Download complete.")
elif not SKIP_RAW_INGESTION:
    print(f"Data exists at {DATA_DIR}")

## Ingest CT and Segmentation Collections

Ingest both CT images and segmentation masks as separate collections in a single RadiObject using the `images` dict API.

In [ ]:
if not SKIP_RAW_INGESTION:
    # Ingest CT images AND segmentation masks in a single call
    radi_raw = RadiObject.from_niftis(
        uri=MSD_LUNG_RAW_URI,
        images={
            "CT": DATA_DIR / "imagesTr",
            "seg": DATA_DIR / "labelsTr",
        },
        validate_alignment=True,  # Ensure CT and seg have matching subjects
        progress=True,
    )
    print(f"\nCreated raw RadiObject: {radi_raw}")
    print(f"CT collection shape: {radi_raw.CT.shape}")
    print(f"seg collection shape: {radi_raw.seg.shape}")
elif uri_exists(MSD_LUNG_RAW_URI):
    radi_raw = RadiObject(MSD_LUNG_RAW_URI)
    print(f"Loaded existing raw: {radi_raw}")
    print(f"Collections: {radi_raw.collection_names}")
else:
    radi_raw = None
    print("Raw data not available - will skip transform step")

## Transform with `map()`

Resample both CT and segmentation to 2mm isotropic spacing. Use linear interpolation for CT and nearest-neighbor for segmentation masks.

In [ ]:
from scipy.ndimage import zoom


def resample_to_2mm(volume: np.ndarray) -> np.ndarray:
    """Resample volume to ~2mm isotropic spacing (linear interpolation)."""
    return zoom(volume, 0.5, order=1)


def resample_to_2mm_nearest(volume: np.ndarray) -> np.ndarray:
    """Resample segmentation mask (nearest neighbor to preserve labels)."""
    return zoom(volume, 0.5, order=0)


if radi_raw is not None and not SKIP_NORMALIZATION:
    # Resample CT with linear interpolation
    print("Resampling CT volumes...")
    ct_vc = radi_raw.CT.map(resample_to_2mm).to_volume_collection(
        uri=f"{MSD_LUNG_URI}/collections/CT",
        progress=True,
    )
    print(f"Created normalized CT: {ct_vc}")

    # Resample segmentation with nearest neighbor
    print("\nResampling segmentation masks...")
    seg_vc = radi_raw.seg.map(resample_to_2mm_nearest).to_volume_collection(
        uri=f"{MSD_LUNG_URI}/collections/seg",
        progress=True,
    )
    print(f"Created normalized seg: {seg_vc}")

    # Create RadiObject group structure and register collections
    tiledb.Group.create(MSD_LUNG_URI, ctx=global_ctx())
    collections_uri = f"{MSD_LUNG_URI}/collections"
    tiledb.Group.create(collections_uri, ctx=global_ctx())

    with tiledb.Group(MSD_LUNG_URI, "w", ctx=global_ctx()) as grp:
        grp.add(collections_uri, name="collections")

    with tiledb.Group(collections_uri, "w", ctx=global_ctx()) as grp:
        grp.add(f"{MSD_LUNG_URI}/collections/CT", name="CT")
        grp.add(f"{MSD_LUNG_URI}/collections/seg", name="seg")

    print(f"\nCreated normalized RadiObject at {MSD_LUNG_URI}")
else:
    print(f"Skipping - normalized data already exists at {MSD_LUNG_URI}")

## Verify Normalized Data

In [ ]:
# Load the normalized RadiObject
radi = RadiObject(MSD_LUNG_URI)

print(f"Subjects: {len(radi)}")
print(f"Collections: {radi.collection_names}")
print(f"CT shape: {radi.CT.shape}")
print(f"seg shape: {radi.seg.shape}")

In [ ]:
# Visualize CT and segmentation overlay
subject_id = list(radi.obs_subject_ids)[0]
ct_vol = radi.loc[subject_id].CT.iloc[0]
seg_vol = radi.loc[subject_id].seg.iloc[0]

# Find slice with tumor
seg_data = seg_vol.to_numpy()
tumor_slices = np.where(seg_data.sum(axis=(0, 1)) > 0)[0]
if len(tumor_slices) > 0:
    mid_z = tumor_slices[len(tumor_slices) // 2]
else:
    mid_z = seg_data.shape[2] // 2

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# CT
axes[0].imshow(ct_vol.axial(z=mid_z).T, cmap="gray", origin="lower")
axes[0].set_title(f"CT (z={mid_z})")
axes[0].axis("off")

# Segmentation
axes[1].imshow(seg_vol.axial(z=mid_z).T, cmap="hot", origin="lower")
axes[1].set_title(f"Segmentation (z={mid_z})")
axes[1].axis("off")

# Overlay
axes[2].imshow(ct_vol.axial(z=mid_z).T, cmap="gray", origin="lower")
mask = seg_vol.axial(z=mid_z).T > 0
axes[2].imshow(np.ma.masked_where(~mask, mask), cmap="Reds", alpha=0.5, origin="lower")
axes[2].set_title(f"Overlay (z={mid_z})")
axes[2].axis("off")

plt.suptitle(f"Subject: {subject_id}")
plt.tight_layout()
plt.show()

**Next:** [06_ml_training.ipynb](./06_ml_training.ipynb) - Train a segmentation model using this data.